* ref: https://www.jianshu.com/p/44a000657eff
# ad_static_feature

In [1]:
import pandas as pd
from constants import *

In [2]:
_data_dir = 'data/testA/'

In [3]:
# 先将所有字段类型设为字符串方便清洗
ad_static_feature = pd.read_csv(_data_dir + 'ad_static_feature.out', sep='\t', header=None, names=HEADER_AD_STATIC,
                               index_col=False,dtype=str, quotechar='\t')
ad_static_feature.head()

,ad_id,create_ts,account_id,product_id,product_class,industry_id,sizes
0,106452,1529958950,22226,16088,13,225,NaN
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
4,457009,1550439402,23614,7447,13,172,NaN


In [4]:
print(len(ad_static_feature))
# 商品id字段可以为空, 其他字段为空则清洗掉
non_na_cols_ad_static = [col for col in ad_static_feature.columns]
ad_static_feature.dropna(axis=0, how='any', thresh=None, subset=non_na_cols_ad_static, inplace=True)
print(len(ad_static_feature))

735911
509252


In [5]:
ad_static_feature.head()

,ad_id,create_ts,account_id,product_id,product_class,industry_id,sizes
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
5,733436,1552977426,22405,31722,5,117,64
6,249105,1552641796,11360,29999,18,145,44


In [6]:
# 注意product_id有多值情况(eg. '6199,28123'), 清洗掉
# FAQ: 商品id不会是多值，文件里面这一列用-1表示的无商品id
ad_static_feature = ad_static_feature[~(ad_static_feature.product_id.str.contains(',', na=False))]  # bad operand type for unary ~: 'float'
print(len(ad_static_feature))

509249


In [7]:
ad_static_feature.head()

,ad_id,create_ts,account_id,product_id,product_class,industry_id,sizes
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
5,733436,1552977426,22405,31722,5,117,64
6,249105,1552641796,11360,29999,18,145,44


In [8]:
# 同样清洗掉industry_id有多值的情况
ad_static_feature.product_id = ad_static_feature.product_id.astype(int)
ad_static_feature = ad_static_feature[~(ad_static_feature.industry_id.str.contains(','))]
print(len(ad_static_feature))
ad_static_feature.industry_id = ad_static_feature.industry_id.astype(int)

502153


In [9]:
print(ad_static_feature[(ad_static_feature.product_id == 0) | (ad_static_feature.industry_id == 0)])
# 广告行业id为-1的情况，测试集中也出现
# ad_static_feature = ad_static_feature[(ad_static_feature.industry_id != -1)]
print(len(ad_static_feature))

Empty DataFrame
Columns: [ad_id, create_ts, account_id, product_id, product_class, industry_id, sizes]
Index: []
502153


In [10]:
# 清洗掉创建时间为0或小于0的广告，？？？
ad_static_feature.create_ts = ad_static_feature.create_ts.astype(int)
ad_static_feature = ad_static_feature[ad_static_feature.create_ts > 0]
print(len(ad_static_feature))

497666


In [11]:
ad_static_feature.ad_id = ad_static_feature.ad_id.astype(int)
ad_static_feature.account_id = ad_static_feature.account_id.astype(int)
ad_static_feature[(ad_static_feature.ad_id == 0) | (ad_static_feature.account_id == 0)]

,ad_id,create_ts,account_id,product_id,product_class,industry_id,sizes


In [12]:
ad_static_feature.dtypes

ad_id             int64
create_ts         int64
account_id        int64
product_id        int64
product_class    object
industry_id       int64
sizes            object
dtype: object

In [13]:
ad_static_feature.product_class = ad_static_feature.product_class.astype(int)
ad_static_feature[(ad_static_feature.product_class == 0)]

,ad_id,create_ts,account_id,product_id,product_class,industry_id,sizes


In [14]:
ad_static_feature.describe()

,ad_id,create_ts,account_id,product_id,product_class,industry_id
count,497666.000000,4.976660e+05,497666.000000,497666.000000,497666.000000,497666.000000
mean,367876.355530,1.550480e+09,14680.788057,11397.578748,9.662569,128.197572
std,212463.741304,4.512075e+06,8618.441474,11334.727152,7.050860,68.086497
min,1.000000,1.428400e+09,1.000000,-1.000000,1.000000,-1.000000
25%,183813.250000,1.550632e+09,7230.000000,-1.000000,3.000000,85.000000
50%,368043.500000,1.551408e+09,14641.000000,7829.000000,8.000000,117.000000
75%,551915.750000,1.552273e+09,22229.000000,21425.000000,18.000000,186.000000
max,736053.000000,1.553007e+09,29742.000000,32929.000000,18.000000,251.000000


In [15]:
ad_static_feature.nunique()

ad_id            497666
create_ts        381620
account_id        22870
product_id        25403
product_class        11
industry_id         233
sizes                65
dtype: int64

In [16]:
# 以上ad_id已经唯一了
ad_static_feature.drop_duplicates(subset='ad_id', keep='last', inplace=True)
ad_static_feature.to_hdf(_data_dir + 'clean_data.h5', 'ad_static_feature')
ad_static_feature.to_csv(_data_dir + 'clean_ad_static_feature.tsv', index=None, sep='\t')

# ad_operation

In [17]:
# TODO：清除201902300000等时间字符串不对的情况，
# TODO：清除ad_static_feature没有的广告id（最后再做，可能ad_static_feature需要重新清洗）
from constants import HEADER_AD_OPERATION
ad_operation = pd.read_csv(_data_dir + 'ad_operation.dat', header=None, sep='\t', names=HEADER_AD_OPERATION,
                          index_col=False, dtype=None, quotechar='\t')
ad_operation.head()

,ad_id,op_time,op_type,op_col,finished_kvs
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90


In [18]:
ad_operation.describe()

,ad_id,op_time,op_type,op_col
count,760866.000000,7.608660e+05,760866.000000,760866.000000
mean,366502.509025,1.717885e+13,1.147152,1.523331
std,210931.328143,7.192564e+12,0.354258,0.855417
min,31.000000,0.000000e+00,1.000000,1.000000
25%,184651.000000,2.019022e+13,1.000000,1.000000
50%,363843.000000,2.019030e+13,1.000000,1.000000
75%,547318.000000,2.019031e+13,1.000000,2.000000
max,736053.000000,2.019032e+13,2.000000,4.000000


In [19]:
# 清洗：去重
print(len(ad_operation))
ad_operation.drop_duplicates(subset=['ad_id', 'op_time', 'op_type', 'op_col'], keep='last', inplace=True)
print(len(ad_operation))

760866
760096


In [20]:
# 清洗：根据赛题，仅保留当天最后修改的值
ad_operation.sort_values(['ad_id', 'op_time', 'op_type', 'op_col'], inplace=True)
ad_operation['op_day'] = ad_operation.op_time // 1000000
ad_operation.head()

,ad_id,op_time,op_type,op_col,finished_kvs,op_day
721281,31,0,2,2,100,0
721282,31,0,2,3,area:11442,0
721283,31,0,2,4,"281474976694272,281474976694272,28147497669427...",0
721284,31,20190313195207,1,1,0,20190313
250461,32,0,2,2,83,0


In [21]:
print(len(ad_operation))
ad_operation.drop_duplicates(subset=['ad_id', 'op_day', 'op_type', 'op_col'], keep='last', inplace=True)
print(len(ad_operation))

760096
494044


In [22]:
ad_operation.duplicated(subset=['ad_id', 'op_day', 'op_col']).any()

False

In [23]:
# 保险起见，还是要再次去重
ad_operation.drop_duplicates(subset=['ad_id', 'op_day', 'op_col'], keep='last', inplace=True)
print(len(ad_operation))

494044


## 将ad_operation转成ad_daily_fea(仅next_day无修改的ad)

In [25]:
# ad_operation['status'] = 
# op_type_map = {1: 'status', 2: 'bid', 3: 'crow', 4: 'exposure_time'}

import numpy as np
ad_operation['status'] = np.where(ad_operation.op_col == 1, ad_operation.finished_kvs, '')
ad_operation['bid'] = np.where(ad_operation.op_col == 2, ad_operation.finished_kvs, '')
ad_operation['crow'] = np.where(ad_operation.op_col == 3, ad_operation.finished_kvs, '')
ad_operation['exposure_time'] = np.where(ad_operation.op_col == 4, ad_operation.finished_kvs, '')
ad_daily_fea = ad_operation[['ad_id', 'op_day', 'status', 'bid', 'crow', 'exposure_time']].groupby(['ad_id', 'op_day']).agg(lambda x: ''.join(x))
ad_daily_fea.unstack(['ad_id', 'op_day'])  # 没有inplace参数，有返回值，如果需要改动应为 ad_daily_fea.unstack(['ad_id', 'op_day'])
ad_daily_fea.head()
# ad_operation_recs = ad_operation.to_dict('recs')
# sorted by ['ad_id', 'op_day', 'op_type', 'op_col']
# 曝光日志为2.17到3.19，有部分2.16的数据
## op_type == 2时op_time为0，则op_day也为0, 需要去ad_static_feature取创建时间
# from common import get_day_list
# day_list = [int(x) for x in get_day_list('20190217', '20190319')]
# # 耗时较长
# def get_ad_status(ad_id, op_day, recs):
#     res = {}
#     for rec in filter(lambda rec: rec.get('ad_id') == ad_id and rec.get('op_day') == op_day ,recs):
#         res[rec.get('op_col')] = rec.get('finished_kvs')
        
#     rec['ad_id'] = ad_id
#     rec['op_day'] = op_day
#     # 保留修改／创建标志？
#     return res
# ad_daily_recs = [get_ad_status(ad_id, op_day, ad_operation_recs)
#                  for ad_id in ad_operation.ad_id.drop_duplicates().tolist()
#                  for op_day in day_list]


status  bid        crow  \
ad_id op_day                             
31    0                100  area:11442   
      20190313      0                    
32    0                 83   area:7572   
      20190221      0   91               
      20190222      0                    

                                                    exposure_time  
ad_id op_day                                                       
31    0         281474976694272,281474976694272,28147497669427...  
      20190313                                                     
32    0         70368475742208,70368475742208,70368475742208,7...  
      20190221                                                     
      20190222

In [26]:
ad_daily_fea.columns

Index([u'status', u'bid', u'crow', u'exposure_time'], dtype='object')

In [27]:
ad_daily_fea[['ad_id', 'op_day']] = ad_daily_fea.index.to_frame()
ad_daily_fea.head()

status  bid        crow  \
ad_id op_day                             
31    0                100  area:11442   
      20190313      0                    
32    0                 83   area:7572   
      20190221      0   91               
      20190222      0                    

                                                    exposure_time  ad_id  \
ad_id op_day                                                               
31    0         281474976694272,281474976694272,28147497669427...     31   
      20190313                                                        31   
32    0         70368475742208,70368475742208,70368475742208,7...     32   
      20190221                                                        32   
      20190222                                                        32   

                  op_day  
ad_id op_day              
31    0                0  
      20190313  20190313  
32    0                0  
      20190221  20190221  
      20190222  20190222

In [28]:
ad_daily_fea.reset_index(drop=True, inplace=True)

In [29]:
ad_daily_fea.head()

,status,bid,crow,exposure_time,ad_id,op_day
0,,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,0
1,0,,,,31,20190313
2,,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,0
3,0,91,,,32,20190221
4,0,,,,32,20190222


In [30]:
ad_daily_fea.describe()

,ad_id,op_day
count,321606.000000,3.216060e+05
mean,367890.071805,1.775173e+07
std,212675.939755,6.579404e+06
min,31.000000,0.000000e+00
25%,182253.000000,2.019022e+07
50%,368484.000000,2.019030e+07
75%,554053.000000,2.019031e+07
max,736053.000000,2.019032e+07


In [31]:
ad_daily_fea.status.unique()

array(['', '0', '1'], dtype=object)

In [32]:
ad_daily_fea.tail()

,status,bid,crow,exposure_time,ad_id,op_day
321601,0,,,,736048,20190317
321602,0,,,,736048,20190319
321603,,150,area:7572,"281474976694272,281474976694272,28147497669427...",736053,0
321604,0,,,,736053,20190314
321605,0,,,,736053,20190315


In [34]:
# 新建广告status默认为1
print(len(ad_daily_fea.ad_id.drop_duplicates()))
print(len(ad_daily_fea[(ad_daily_fea['op_day']==0)&(ad_daily_fea['status']=='')]))
ad_daily_fea.loc[ad_daily_fea[(ad_daily_fea['op_day']==0)&(ad_daily_fea['status']=='')].index,['status']] = '1'

38843
37321


In [35]:
ad_daily_fea.head()

,status,bid,crow,exposure_time,ad_id,op_day
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,0
1,0,,,,31,20190313
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,0
3,0,91,,,32,20190221
4,0,,,,32,20190222


In [36]:
ad_daily_fea.status.drop_duplicates()

0     1
1     0
22     
Name: status, dtype: object

In [37]:
ad_daily_fea.head(25)

,status,bid,crow,exposure_time,ad_id,op_day
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,0
1,0,,,,31,20190313
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,0
3,0,91,,,32,20190221
4,0,,,,32,20190222
5,0,,,,32,20190223
6,0,,,,32,20190224
7,0,90,,,32,20190225
8,0,,,,32,20190226
9,0,96,,,32,20190227


In [47]:
ad_daily_fea.replace('', np.NaN, inplace=True)  # 设置空字符串为NaN方便之后识别处理
ad_init_fea = ad_daily_fea.groupby('ad_id').first()
ad_init_fea.head()

,status,bid,crow,exposure_time,op_day
ad_id,,,,,
31,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",0
32,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",0
69,1,180,area:7572,"281474976694272,281474976694272,28147497669427...",0
84,1,100,area:6410,"4398045462528,4398045462528,4398045462528,4398...",0
85,1,70,"age:217,601,202,837,942,638,287,5,394,347,731,...","281474976694272,281474976694272,28147497669427...",0


In [48]:
na_ad = ad_init_fea[ad_init_fea.isna().any(axis=1)]
na_ad.head()

,status,bid,crow,exposure_time,op_day
ad_id,,,,,
202,0,NaN,NaN,NaN,0
247,0,NaN,NaN,NaN,0
430,0,NaN,NaN,NaN,0
694,0,NaN,NaN,NaN,0
700,0,NaN,NaN,NaN,0


In [49]:
# 对ad_daily_fea做清理
print(len(ad_daily_fea))
ad_daily_fea = ad_daily_fea[~(ad_daily_fea.ad_id.isin(na_ad.index))]
print(len(ad_daily_fea))

321606
320084


In [50]:
ad_daily_fea.head()

,status,bid,crow,exposure_time,ad_id,op_day
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,0
1,0,NaN,NaN,NaN,31,20190313
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,0
3,0,91,NaN,NaN,32,20190221
4,0,NaN,NaN,NaN,32,20190222


In [51]:
# 其他空字符串由历史继承, NOTE: 暂不去除无效op_day(0230)！！！ 因为觉得这个时间改动的可能会影响后续的值， 但发现其实均是自动的广告失效操作
# 暂不去除静态广告数据中没有的ad_id
ad_fea_op_by_day = ad_daily_fea.fillna(method='pad')
ad_fea_op_by_day.head()

,status,bid,crow,exposure_time,ad_id,op_day
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,0
1,0,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,20190313
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,0
3,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190221
4,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190222


In [66]:
problem_ad_id = ad_fea_op_by_day[(ad_fea_op_by_day.op_day > 20190228) & (ad_fea_op_by_day.op_day < 20190301)]['ad_id']
ad_fea_op_by_day[ad_fea_op_by_day.ad_id.isin(problem_ad_id)]

,status,bid,crow,exposure_time,ad_id,op_day
177,1,300,all,"17592185782272,17592185782272,17592185782272,1...",415,0
178,0,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190217
179,1,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190218
180,0,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190222
181,1,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190225
182,0,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190228
183,0,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190230
184,1,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190301
185,1,600,all,"17592185782272,17592185782272,17592185782272,1...",415,20190305
186,0,600,all,"17592185782272,17592185782272,17592185782272,1...",415,20190313


In [53]:
ad_fea_op_by_day.to_hdf(_data_dir + 'clean_data.h5', 'ad_fea_op_by_day')
ad_fea_op_by_day.to_csv(_data_dir + 'ad_fea_op_by_day.tsv', index=None, sep='\t')

# 构造待预测的sample
1. merge并转换op_day，确认时间正常（TODO：实际应在之前就判断好了！！！）
2. 去掉后一天有改动过的

In [63]:
ad_sample = ad_fea_op_by_day.merge(ad_static_feature, on=['ad_id'])
ad_sample.head()

,status,bid,crow,exposure_time,ad_id,op_day,create_ts,account_id,product_id,product_class,industry_id,sizes
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,0,1552432970,12577,18683,13,224,40
1,0,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,20190313,1552432970,12577,18683,13,224,40
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,0,1543563617,18752,32534,13,136,40
3,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190221,1543563617,18752,32534,13,136,40
4,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190222,1543563617,18752,32534,13,136,40


In [64]:
# 注意，where第二个参数为条件为负时的取值！！！
ad_sample.op_day = ad_sample.op_day.where(ad_sample.op_day > 0,
                                          pd.to_datetime(ad_sample.create_ts, unit='s').dt.strftime('%Y%m%d').astype(int)
                                         )  # 不能使用inplace=True， 即使前面没有再赋值
ad_sample['op_date'] = pd.to_datetime(ad_sample.op_day.astype(str), errors='coerce', format='%Y%m%d')
ad_sample.head()

,status,bid,crow,exposure_time,ad_id,op_day,create_ts,account_id,product_id,product_class,industry_id,sizes,op_date
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,20190312,1552432970,12577,18683,13,224,40,2019-03-12
1,0,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,20190313,1552432970,12577,18683,13,224,40,2019-03-13
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20181130,1543563617,18752,32534,13,136,40,2018-11-30
3,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190221,1543563617,18752,32534,13,136,40,2019-02-21
4,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190222,1543563617,18752,32534,13,136,40,2019-02-22


In [65]:
#pd.isnull also works for NaTs
ad_sample[ad_sample.op_date.isnull()]

,status,bid,crow,exposure_time,ad_id,op_day,create_ts,account_id,product_id,product_class,industry_id,sizes,op_date
137,0,300,all,"17592185782272,17592185782272,17592185782272,1...",415,20190230,1547789504,17240,-1,1,117,1,NaT
218,0,35,area:7572,"281474976694272,281474976694272,28147497669427...",701,20190230,1545597363,24664,4422,13,84,34,NaT
1182,0,180,"age:1,347,787,753,601,522,202,229,731,333,393,...","281474976710655,281474976710655,28147497671065...",3735,20190230,1550564433,20223,-1,1,244,64,NaT
1231,0,200,"age:217,601,79,202,837,942,638,394,347,731,739...","4397038829568,4397038829568,4397038829568,4397...",3849,20190230,1541296966,18493,28846,13,12,34,NaT
1364,0,94,area:3269,"281200098803712,281200098803712,28120009880371...",4160,20190230,1545205217,20179,29401,13,186,30,NaT
1851,0,96,area:7572,"17317308137472,17317308137472,17317308137472,1...",5682,20190230,1540524103,26937,32725,13,136,30,NaT
2111,0,200,"age:217,601,79,202,837,942,638,394,347,731,739...","281474976694272,281474976694272,28147497669427...",6105,20190230,1541018032,5204,24563,13,84,34,NaT
2299,0,380,"area:11524,9679,6432,4224,6410,12995,12222,917...","281474976710655,281474976710655,28147497671065...",6560,20190230,1547806474,13105,-1,1,38,1,NaT
2993,0,78,"age:1,400,787,217,202,229,731,739,333,393,988,...","281474976694272,281474976694272,28147497669427...",8428,20190230,1550241767,13896,-1,1,217,64,NaT
3547,0,100,"age:217,601,79,202,837,942,638,394,347,731,739...","281474976694272,281474976694272,28147497669427...",9970,20190230,1540981895,9436,20243,13,146,34,NaT


In [67]:
# 但不适用dropna...
print(len(ad_sample))
ad_sample.dropna(axis=0, how='any', thresh=None, subset=non_na_cols_ad_static, inplace=True)
print(len(ad_sample))

258137
258137


In [68]:
ad_sample = ad_sample[~(ad_sample.op_date.isnull())]
print(len(ad_sample))

257169


In [70]:
ad_sample.head()

,status,bid,crow,exposure_time,ad_id,op_day,create_ts,account_id,product_id,product_class,industry_id,sizes,op_date,id
0,1,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,20190312,1552432970,12577,18683,13,224,40,2019-03-12,0
1,0,100,area:11442,"281474976694272,281474976694272,28147497669427...",31,20190313,1552432970,12577,18683,13,224,40,2019-03-13,1
2,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20181130,1543563617,18752,32534,13,136,40,2018-11-30,2
3,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190221,1543563617,18752,32534,13,136,40,2019-02-21,3
4,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",32,20190222,1543563617,18752,32534,13,136,40,2019-02-22,4


In [153]:
# todo: 0230失效后下一天怎么办？
ad_sample['altered'] = 1
from common import get_day_list
date_list = get_day_list('20190217', '20190319')
ad_daily_recs = [{'op_date': op_date, 'ad_id': ad_id}
                 for ad_id in ad_sample.ad_id.drop_duplicates().tolist()
                 for op_date in date_list]

ad_daily_sample = pd.DataFrame(ad_daily_recs).merge(ad_sample, on=['ad_id', 'op_date'], how='outer')  # ! 不能使用left，会丢失创建数据

In [156]:
ad_daily_sample[ad_daily_sample.ad_id == 32]

,ad_id,op_date,status,bid,crow,exposure_time,op_day,create_ts,account_id,product_id,product_class,industry_id,sizes,altered
32,32,2019-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,32,2019-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,32,2019-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,32,2019-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,32,2019-02-21,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",20190221.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0
37,32,2019-02-22,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",20190222.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0
38,32,2019-02-23,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",20190223.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0
39,32,2019-02-24,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",20190224.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0
40,32,2019-02-25,0,90,area:7572,"70368475742208,70368475742208,70368475742208,7...",20190225.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0
41,32,2019-02-26,0,90,area:7572,"70368475742208,70368475742208,70368475742208,7...",20190226.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0


In [157]:
# !!! merge后需要重新排序
ad_daily_sample.sort_values(['ad_id', 'op_date'], inplace=True)
ad_daily_sample.altered = ad_daily_sample.altered.fillna(0)
ad_daily_sample[ad_daily_sample.ad_id == 32].head()

,ad_id,op_date,status,bid,crow,exposure_time,op_day,create_ts,account_id,product_id,product_class,industry_id,sizes,altered
927802,32,2018-11-30,1,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",20181130.0,1.543564e+09,18752.0,32534.0,13.0,136.0,40,1.0
32,32,2019-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
33,32,2019-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
34,32,2019-02-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
35,32,2019-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [158]:
# 注意，为了避免将尚未创建的广告的NaN值设为上一个，需要groupby后再fillna
print(len(ad_daily_sample))
ad_date_valid = ad_daily_sample.groupby('ad_id').fillna(method='pad')  # 不可使用downcast, 否则dropna出错
print(len(ad_date_valid))
ad_date_valid.head(30).tail(10)

940425
940425


,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status
20,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2019-03-09,NaN,NaN,NaN,NaN,NaN
21,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2019-03-10,NaN,NaN,NaN,NaN,NaN
22,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2019-03-11,NaN,NaN,NaN,NaN,NaN
23,12577.0,1.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-12,20190312.0,13.0,18683.0,40,1
24,12577.0,1.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-13,20190313.0,13.0,18683.0,40,0
25,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-14,20190313.0,13.0,18683.0,40,0
26,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-15,20190313.0,13.0,18683.0,40,0
27,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-16,20190313.0,13.0,18683.0,40,0
28,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-17,20190313.0,13.0,18683.0,40,0
29,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-18,20190313.0,13.0,18683.0,40,0


In [160]:
ad_date_valid['ad_id'] = ad_daily_sample.ad_id.copy()
ad_date_valid.reset_index(drop=True, inplace=True)  # 此后才可以进行dropna
ad_date_valid.dropna(axis=0, how='any', thresh=None, subset=non_na_cols_ad_static, inplace=True)
print(len(ad_date_valid))

697997


In [161]:
ad_date_valid.head()

,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id
23,12577.0,1.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-12,20190312.0,13.0,18683.0,40,1,31
24,12577.0,1.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-13,20190313.0,13.0,18683.0,40,0,31
25,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-14,20190313.0,13.0,18683.0,40,0,31
26,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-15,20190313.0,13.0,18683.0,40,0,31
27,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-16,20190313.0,13.0,18683.0,40,0,31


In [163]:
# merge后类型转变了，保存文件重新读取后可以识别为准确的类型
ad_date_valid.to_csv(_data_dir + 'ad_date_valid.tsv', index=None, sep='\t')
# 条件：同一ad_id下一天无修改操作 或 下一天记录为另外的ad_id(包括最后一条的情况，为空)
# shift param `fill_value` supported in version 0.24.0，因此最好升级
train_sample = ad_date_valid[((ad_date_valid.altered.shift(-1)  == 0) & (ad_date_valid.ad_id.shift(-1) == ad_date_valid.ad_id))
                            | (ad_date_valid.ad_id.shift(-1) != ad_date_valid.ad_id)]  # 最后一条也可以加进去
print(len(train_sample))
train_sample.head()

469610


,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id
24,12577.0,1.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-13,20190313.0,13.0,18683.0,40,0,31
25,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-14,20190313.0,13.0,18683.0,40,0,31
26,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-15,20190313.0,13.0,18683.0,40,0,31
27,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-16,20190313.0,13.0,18683.0,40,0,31
28,12577.0,0.0,100,1.552433e+09,area:11442,"281474976694272,281474976694272,28147497669427...",224.0,2019-03-17,20190313.0,13.0,18683.0,40,0,31


In [164]:
print(len(train_sample[train_sample.status == 1]))  # wrong: type is object!

0


In [165]:
ad_date_valid[ad_date_valid.ad_id == 32].head()

,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id
32,18752.0,1.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2018-11-30,20181130.0,13.0,32534.0,40,1,32
33,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-17,20181130.0,13.0,32534.0,40,1,32
34,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-18,20181130.0,13.0,32534.0,40,1,32
35,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-19,20181130.0,13.0,32534.0,40,1,32
36,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-20,20181130.0,13.0,32534.0,40,1,32


In [166]:
train_sample[train_sample.ad_id == 32]

,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id
32,18752.0,1.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2018-11-30,20181130.0,13.0,32534.0,40,1,32
33,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-17,20181130.0,13.0,32534.0,40,1,32
34,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-18,20181130.0,13.0,32534.0,40,1,32
35,18752.0,0.0,83,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-02-19,20181130.0,13.0,32534.0,40,1,32
46,18752.0,1.0,97,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-03-02,20190302.0,13.0,32534.0,40,0,32
63,18752.0,1.0,87,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-03-19,20190319.0,13.0,32534.0,40,0,32
64,18752.0,0.0,87,1.543564e+09,area:7572,"70368475742208,70368475742208,70368475742208,7...",136.0,2019-03-20,20190319.0,13.0,32534.0,40,0,32


In [167]:
train_sample.status.describe()

count     469610
unique         2
top            0
freq      293930
Name: status, dtype: object

In [168]:
print(len(train_sample[train_sample.status == '1']))

175680


In [169]:
train_sample = train_sample[train_sample.status == '1']

In [171]:
print(len(train_sample[train_sample.op_date.dt.strftime('%Y%m%d') > '20190215']))

167168


In [172]:
# 额，其实可以构造2.16最终的数据来预测（之前已新建且2.17无修改的情况），以下只包含了26日新建的广告，可扩充
train_sample = train_sample[train_sample.op_date.dt.strftime('%Y%m%d') > '20190215']
train_sample[train_sample.op_date.dt.strftime('%Y%m%d') == '20190216']

,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,status,ad_id
20009,2329.0,1.0,90,1.550301e+09,area:9164,"281474976694272,281474976694272,28147497669427...",21.0,2019-02-16,20190216.0,13.0,2046.0,30,1,14705
28826,9721.0,1.0,75,1.550280e+09,area:7572,"281474976694272,281474976694272,28147497669427...",84.0,2019-02-16,20190216.0,13.0,3473.0,34,1,21612
99478,25057.0,1.0,118,1.550330e+09,area:7572,"281474976694272,281474976694272,28147497669427...",21.0,2019-02-16,20190216.0,13.0,24586.0,30,1,75508
100915,21217.0,1.0,94,1.550358e+09,"age:217,601,79,202,837,942,638,394,347,731,739...","53034521198592,53034521198592,53034521198592,5...",146.0,2019-02-16,20190216.0,13.0,7087.0,1,1,76459
153156,16882.0,1.0,69,1.550296e+09,"age:217,601,79,202,837,942,638,394,347,731,739...","281474976694272,281474976694272,28147497669427...",146.0,2019-02-16,20190216.0,13.0,21775.0,30,1,117464
166417,10969.0,1.0,61,1.550294e+09,"age:787,753,601,202,229,608,479,394|gender:2|a...","281474976710655,281474976710655,28147497671065...",170.0,2019-02-16,20190216.0,1.0,-1.0,64,1,128311
198581,12193.0,1.0,120,1.550287e+09,"age:347,787,753,601,522,202,229,333,741,819,47...","281474976694272,281474976694272,28147497669427...",170.0,2019-02-16,20190216.0,1.0,-1.0,64,1,151829
201291,14521.0,1.0,86,1.550325e+09,area:3269,"281474976694272,281474976694272,28147497669427...",84.0,2019-02-16,20190216.0,13.0,31764.0,30,1,153738
201682,12162.0,1.0,31,1.550299e+09,"age:217,601,79,202,837,942,638,394,347,731,739...","1095216660480,1095216660480,1095216660480,1095...",146.0,2019-02-16,20190216.0,13.0,22873.0,1,1,153961
209499,16882.0,1.0,100,1.550296e+09,"age:217,601,79,202,837,942,638,394,347,731,739...","281474976694272,281474976694272,28147497669427...",146.0,2019-02-16,20190216.0,13.0,21775.0,1,1,159995


In [173]:
train_sample.to_hdf(_data_dir + 'clean_data.h5', 'train_sample')
train_sample.to_csv(_data_dir + 'train_sample.tsv', index=None, sep='\t')